In [63]:
import pandas as pd
import numpy as np
from scipy.stats import poisson
from scipy.stats import geom
from scipy.stats import binom

In [64]:
%cd D:\StonyBrook\Study\Prob&Stats CSE544\Project

D:\StonyBrook\Study\Prob&Stats CSE544\Project


In [65]:
data = pd.read_csv('7.csv')

In [66]:
## converting date column to datetime data type ##
data['Date'] = pd.to_datetime(data['Date'])

In [67]:
data

,Date,IA confirmed cumulative,ID confirmed cumulative,IA deaths cumulative,ID deaths cumulative,IA confirmed,ID confirmed,IA deaths,ID deaths
0,2020-01-22,0,0,0,0,NaN,NaN,NaN,NaN
1,2020-01-23,0,0,0,0,0.0,0.0,0.0,0.0
2,2020-01-24,0,0,0,0,0.0,0.0,0.0,0.0
3,2020-01-25,0,0,0,0,0.0,0.0,0.0,0.0
4,2020-01-26,0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
433,2021-03-30,349742,179429,5726,1956,141.0,0.0,7.0,0.0
434,2021-03-31,350840,180536,5744,1962,1098.0,1107.0,18.0,6.0
435,2021-01-04,351651,180897,5744,1963,811.0,361.0,0.0,1.0
436,2021-02-04,352262,181181,5752,1966,611.0,284.0,8.0,3.0


In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date                     438 non-null    datetime64[ns]
 1   IA confirmed cumulative  438 non-null    int64         
 2   ID confirmed cumulative  438 non-null    int64         
 3   IA deaths cumulative     438 non-null    int64         
 4   ID deaths cumulative     438 non-null    int64         
 5   IA confirmed             437 non-null    float64       
 6   ID confirmed             437 non-null    float64       
 7   IA deaths                437 non-null    float64       
 8   ID deaths                437 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(4)
memory usage: 30.9 KB


In [69]:
def MME_Poisson(X):
    estimate = np.mean(X)
    return estimate

def CDF_Poisson(lambda_, x):
    prob = poisson.cdf(x, lambda_)
    return prob

def MME_Geometric(X):
    sample_mean = np.mean(X)
    estimate = 1/sample_mean
    
    return estimate

def CDF_Geometric(p,x):
    prob = geom.cdf(x, p)
    return prob

def MME_Binomial(X):
    mean = np.mean(X)
    var = np.var(X)
    estimate_p = 1 - (var/mean)
    estimate_n = mean/estimate_p
    
    #print(mean,var)
    
    return estimate_p,estimate_n

def CDF_Binomial(params,x):
    prob = binom.cdf(x, params[0], params[1])
    return prob

In [70]:
def get_eCDF(X):
    n = len(X)
    Srt = sorted(X)
    delta = .1
    X = []
    Y = [0]
    for i in range(0,n):
        X = X + [Srt[i]]
        Y = Y + [Y[len(Y)-1]+(1/n)]
    Y = Y + [1]
        
    return X,Y


def KS_Test_1_sample(X1,Y1, CDF_function, parameter):
    tot_max = -1
        
    Table = np.zeros((len(X1),4))
    for i in range(len(Table)):
        Table[i,0] = Y1[i]
        Table[i,1] = Y1[i+1]
        F_x = CDF_function(parameter, X1[i])
        Table[i,2] = abs(Table[i,0] - F_x)
        Table[i,3] = abs(Table[i,1] - F_x)
        cmax = max(Table[i,2], Table[i,3])
        if cmax > tot_max:
            tot_max = cmax
        
        
    return tot_max
    

def KS_test_2_sample(X1,Y1, X2,Y2):
    Table = np.zeros((len(X1),6))
    tot_max = -1
    for i in range(len(Table)):
        Table[i,0] = Y1[i]
        Table[i,1] = Y1[i+1]
        index1 = [idx for idx, val in enumerate(X2) if val >= X1[i]]
        index2 = [idx for idx, val in enumerate(X2) if val < X1[i]]
        if index1 == []:
            Table[i,3] = 1
        else :    
            Table[i,3] = Y2[index1[0]]
        if index2 == []:
            Table[i,2] = 0
        else:
            Table[i,2] = Y2[index2[-1]]
        #print(index1, index2)
        
        #Table[i,3] = Y2[index1[0]]
        Table[i,4] = abs( Table[i,0] - Table[i,2])
        Table[i,5] = abs(Table[i,1] - Table[i,3])
        cmax = max(Table[i,4], Table[i,5])
        if cmax > tot_max:
            tot_max = cmax
            x1_max = X1[i]
            y1_max = Table[i,0]
            y2_max = Table[i,2]
    
    return tot_max

In [71]:
def get_Ti(n_perm, data, n1):
    T = []
    for i in range(n_perm):
        permute = np.random.permutation(len(data))
        D1 = data[permute[:n1]]
        D2 = data[permute[n1:]]
        T.append(abs(np.mean(D1) - np.mean(D2)))
    
    return np.array(T)

def get_p_value(T,T_obs):
    count = np.sum(T > T_obs)
    p_val = count/len(T)
    return p_val

In [72]:
###### Getting Oct- Dec 2020 data ######
start_date, end_date = '2020-10-01', '2020-12-31'
condition = (data['Date'] >= start_date) & (data['Date'] <= end_date)
oct_dec_data = data.loc[condition]

In [73]:
################## 1- sample KS test ####################

In [74]:
###### Tests for IA confirmed cases with ID confirmed cases ######

handle = 'IA confirmed'
test_handle = 'ID confirmed'
# Obtaining MME for IA confirmed cases 

# Assuming Poisson distribution 
CDF_dist = CDF_Poisson

print('####### Poisson Distribution ########')
lambda_ = MME_Poisson(oct_dec_data[handle])
print(' Poisson parameter lambda : ', lambda_)


# KS-test to be performed on test_handle

# Obtaining eCDF for test_handle
test_handle_data = oct_dec_data[test_handle]

test_X, test_Y = get_eCDF(test_handle_data)

KS_value = KS_Test_1_sample(test_X, test_Y, CDF_dist, lambda_ )

print(' KS statistic : ', KS_value)

# Critical threshold is 0.05

# Reject Null hypothesis

# Assuming Geometric distribution 

print('####### Geometric Distribution ########')
p = MME_Geometric(oct_dec_data[handle])
CDF_dist = CDF_Geometric

print(' Geometric parameter : ', p)

KS_value = KS_Test_1_sample(test_X, test_Y, CDF_dist, p )

print(' KS statistic : ', KS_value)

# Critical threshold is 0.05

# Reject Null hypothesis


# Assuming Binomial distribution
print('####### Binomial Distribution ########')
n,p = MME_Binomial(oct_dec_data[handle])
CDF_dist = CDF_Binomial

print(' Binomial parameters(n,p) : ', n,p)

KS_value = KS_Test_1_sample(test_X, test_Y, CDF_dist, [n,p] )

print(' KS statistic : ', KS_value)


####### Poisson Distribution ########
 Poisson parameter lambda :  1584.4606741573034
 KS statistic :  0.8089299381535394
####### Geometric Distribution ########
 Geometric parameter :  0.0006311295801215457
 KS statistic :  0.2710049785002102
####### Binomial Distribution ########
 Binomial parameters(n,p) :  -1200.876072476081 -1.319420638376374
 KS statistic :  0.9887640449438202


In [75]:
###### Tests for IA death cases with ID death cases ######

handle = 'IA deaths'
test_handle = 'ID deaths'
# Obtaining MME for IA confirmed cases 

# Assuming Poisson distribution 
CDF_dist = CDF_Poisson

print('####### Poisson Distribution ########')
lambda_ = MME_Poisson(oct_dec_data[handle])
print(' Poisson parameter lambda : ', lambda_)


# KS-test to be performed on test_handle

# Obtaining eCDF for test_handle
test_handle_data = oct_dec_data[test_handle]

test_X, test_Y = get_eCDF(test_handle_data)

KS_value = KS_Test_1_sample(test_X, test_Y, CDF_dist, lambda_ )

print(' KS statistic : ', KS_value)

# Critical threshold is 0.05

# Reject Null hypothesis

# Assuming Geometric distribution 

print('####### Geometric Distribution ########')
p = MME_Geometric(oct_dec_data[handle])
CDF_dist = CDF_Geometric

print(' Geometric parameter : ', p)

KS_value = KS_Test_1_sample(test_X, test_Y, CDF_dist, p )

print(' KS statistic : ', KS_value)

# Critical threshold is 0.05

# Reject Null hypothesis


# Assuming Binomial distribution
print('####### Binomial Distribution ########')
n,p = MME_Binomial(oct_dec_data[handle])
CDF_dist = CDF_Binomial

print(' Binomial parameters(n,p) : ', n,p)

KS_value = KS_Test_1_sample(test_X, test_Y, CDF_dist, [n,p] )

print(' KS statistic : ', KS_value)



####### Poisson Distribution ########
 Poisson parameter lambda :  24.303370786516854
 KS statistic :  0.7477959863659818
####### Geometric Distribution ########
 Geometric parameter :  0.041146555709662504
 KS statistic :  0.37338611456179815
####### Binomial Distribution ########
 Binomial parameters(n,p) :  -136.5731895463542 -0.17795125725073635
 KS statistic :  1.0


In [76]:
######## KS 2-sample Test ##########

In [77]:
###### Tests for IA confirmed cases with ID confirmed cases ######

handle = 'IA confirmed'
test_handle = 'ID confirmed'

# Obtaining eCDF for handles
handle_data = oct_dec_data[handle]

test_handle_data = oct_dec_data[test_handle]

X1, Y1 = get_eCDF(handle_data)
X2, Y2 = get_eCDF(test_handle_data)


KS_value = KS_test_2_sample(X1,Y1, X2,Y2)

print(' KS statistic : ', KS_value)

# Reject Null Hypothesis

 KS statistic :  0.28089887640449507


In [78]:
###### Tests for IA death cases with ID death cases ######

handle = 'IA deaths'
test_handle = 'ID deaths'

# Obtaining eCDF for handles
handle_data = oct_dec_data[handle]

test_handle_data = oct_dec_data[test_handle]

X1, Y1 = get_eCDF(handle_data)
X2, Y2 = get_eCDF(test_handle_data)


KS_value = KS_test_2_sample(X1,Y1, X2,Y2)

print(' KS statistic : ', KS_value)

# Reject Null Hypothesis

 KS statistic :  0.23595505617977586


In [79]:
################## Permutation test ####################

In [80]:
###### Test for IA confirmed cases with ID confirmed cases ######


handle = 'IA confirmed'
test_handle = 'ID confirmed'

# Obtaining eCDF for handles
handle_data = oct_dec_data[handle]

test_handle_data = oct_dec_data[test_handle]

T_obs = np.abs(np.mean(handle_data) - np.mean(test_handle_data))

print(" T_observed is : " ,T_obs)


total_data = np.concatenate((np.array(handle_data) , np.array(test_handle_data)))

T_i = get_Ti(1000, total_data, len(handle_data))

p = get_p_value(T_i, T_obs)

print(' p statistic : ', p)

# Reject Null Hypothesis

 T_observed is :  758.3595505617977
 p statistic :  0.0


In [81]:
###### Test for IA death cases with ID death cases ######


handle = 'IA deaths'
test_handle = 'ID deaths'

# Obtaining eCDF for handles
handle_data = oct_dec_data[handle]

test_handle_data = oct_dec_data[test_handle]

T_obs = np.abs(np.mean(handle_data) - np.mean(test_handle_data))

print(" T_observed is : " ,T_obs)


total_data = np.concatenate((np.array(handle_data) , np.array(test_handle_data)))

T_i = get_Ti(1000, total_data, len(handle_data))

p = get_p_value(T_i, T_obs)

print(' p statistic : ', p)

# Reject Null Hypothesis

 T_observed is :  16.280898876404493
 p statistic :  0.0
